In [2]:
%load_ext autoreload
# %autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import logging
from sys import stdout
from datetime import datetime
import numpy as np
import os

logging.basicConfig(
    level=logging.INFO,
    format="[%(asctime)s][%(levelname)s] %(message)s",
    stream=stdout,
    datefmt="%m-%d %H:%M:%S",
)

logging.info(f"{os.getlogin()}| {datetime.now()}")

[02-08 08:25:10][INFO] Johann| 2025-02-08 08:25:10.927802


In [4]:
# ####################################################################
def gauss_jordan(Ab: np.ndarray) -> np.ndarray:
    """Resuelve un sistema de ecuaciones lineales mediante el método de Gauss-Jordan.

    ## Parameters

    ``Ab``: matriz aumentada del sistema de ecuaciones lineales. Debe ser de tamaño n-by-(n+1), donde n es el número de incógnitas.

    ## Return

    ``solucion``: vector con la solución del sistema de ecuaciones lineales.

    """
    if not isinstance(Ab, np.ndarray):
        logging.debug("Convirtiendo A a numpy array.")
        Ab = np.array(Ab, dtype=float)
    assert Ab.shape[0] == Ab.shape[1] - 1, "La matriz A debe ser de tamaño n-by-(n+1)."
    n = Ab.shape[0]

    for i in range(0, n):  # loop por columna

        # --- encontrar pivote
        p = None  # default, first element
        for pi in range(i, n):
            if Ab[pi, i] == 0:
                # must be nonzero
                continue

            if p is None:
                # first nonzero element
                p = pi
                continue

            if abs(Ab[pi, i]) < abs(Ab[p, i]):
                p = pi

        if p is None:
            # no pivot found.
            raise ValueError("No existe solución única.")

        if p != i:
            # swap rows
            logging.debug(f"Intercambiando filas {i} y {p}")
            _aux = Ab[i, :].copy()
            Ab[i, :] = Ab[p, :].copy()
            Ab[p, :] = _aux

        # --- Eliminación: loop por fila
        for j in range(n):
            if i == j:
                continue
            m = Ab[j, i] / Ab[i, i]
            Ab[j, i:] = Ab[j, i:] - m * Ab[i, i:]

        logging.info(f"\n{Ab}")

    if Ab[n - 1, n - 1] == 0:
        raise ValueError("No existe solución única.")

    # --- Sustitución hacia atrás
    solucion = np.zeros(n)

    for i in range(n - 1, -1, -1):
        solucion[i] = Ab[i, -1] / Ab[i, i]

    return solucion

In [5]:
%autoreload 2
from src import matriz_aumentada

A = [
    [1, 2, 3, 4],
    [2, 5, 6, 7],
    [3, 6, 8, 9],
    [4, 7, 9, 10],
]
b = [1, -2, 3, 4]

Ab = matriz_aumentada(A, b)
gauss_jordan(Ab)

[02-08 08:25:20][INFO] 2025-02-08 08:25:20.580596
[02-08 08:25:20][INFO] Johann| 2025-02-08 08:25:20.582510
[02-08 08:25:20][INFO] 2025-02-08 08:25:20.584035
[02-08 08:25:20][INFO] Johann| 2025-02-08 08:25:20.585697
[02-08 08:25:20][INFO] 
[[ 1.  2.  3.  4.  1.]
 [ 0.  1.  0. -1. -4.]
 [ 0.  0. -1. -3.  0.]
 [ 0. -1. -3. -6.  0.]]
[02-08 08:25:20][INFO] 
[[ 1.  0.  3.  6.  9.]
 [ 0.  1.  0. -1. -4.]
 [ 0.  0. -1. -3.  0.]
 [ 0.  0. -3. -7. -4.]]
[02-08 08:25:20][INFO] 
[[ 1.  0.  0. -3.  9.]
 [ 0.  1.  0. -1. -4.]
 [ 0.  0. -1. -3.  0.]
 [ 0.  0.  0.  2. -4.]]
[02-08 08:25:20][INFO] 
[[ 1.  0.  0.  0.  3.]
 [ 0.  1.  0.  0. -6.]
 [ 0.  0. -1.  0. -6.]
 [ 0.  0.  0.  2. -4.]]


array([ 3., -6.,  6., -2.])

# Resolver

In [32]:
def inv_matrix(A: list) -> list:
    """Inversión de una matriz cuadrada mediante método de Gauss-Jordan con selección de pivote.
    
    ## Parameters
    ``A``: matriz cuadrada de tamaño n x n representada como una lista de listas.

    ## Return
    ``A_inv``: matriz inversa de A representada como una lista de listas.
    """
    # Verificar si la matriz es invertible calculando su determinante
    def determinant(matrix):
        """Calcula el determinante de una matriz cuadrada usando el método de Laplace."""
        n = len(matrix)
        if n == 1:
            return matrix[0][0]
        if n == 2:
            return matrix[0][0] * matrix[1][1] - matrix[0][1] * matrix[1][0]
        det = 0
        for col in range(n):
            submatrix = [row[:col] + row[col+1:] for row in matrix[1:]]
            cofactor = (-1) ** col * matrix[0][col] * determinant(submatrix)
            det += cofactor
        return det
    
    if determinant(A) == 0:
        raise ValueError("La matriz no es invertible (determinante = 0).")
    
    n = len(A)
    
    # Crear la matriz identidad de tamaño n x n
    I = [[1 if i == j else 0 for j in range(n)] for i in range(n)]
    
    # Convertir A en una copia mutable
    A = [row[:] for row in A]
    
    for i in range(n):
        # Encontrar el mejor pivote (selección de pivote parcial)
        max_row = max(range(i, n), key=lambda r: abs(A[r][i]))
        if A[max_row][i] == 0:
            raise ValueError("La matriz no es invertible.")
        
        # Intercambiar filas si es necesario
        if max_row != i:
            A[i], A[max_row] = A[max_row], A[i]
            I[i], I[max_row] = I[max_row], I[i]
        
        # Normalizar la fila dividiendo por el pivote
        pivot = A[i][i]
        for j in range(n):
            A[i][j] /= pivot
            I[i][j] /= pivot
        
        # Hacer ceros en las otras filas
        for k in range(n):
            if k == i:
                continue
            factor = A[k][i]
            for j in range(n):
                A[k][j] -= factor * A[i][j]
                I[k][j] -= factor * I[i][j]
    
    return I

In [27]:
# Función para imprimir la matriz de forma legible
def print_matrix(matrix):
    for row in matrix:
        print("  ".join(f"{val:.4f}" for val in row))

## Ejemplos
* Ejemplo 1

In [28]:
# La matriz A =
A = [
    [1, 2, 3, 4],
    [2, 5, 6, 7],
    [3, 6, 8, 9],
    [4, 7, 9, 10],
]
# tiene como inversa
# A_inv =[[ 0.5, -0.5, -1.5,  1.5],
#        [-0.5,  1.5, -1.5,  0.5],
#        [-1.5, -1.5,  3.5, -1.5],
#        [ 1.5,  0.5, -1.5,  0.5]]
# Imprimir la matriz inversa de A
A_inv = inv_matrix(A)
print("Matriz Inversa A⁻¹:")
print_matrix(A_inv)

Matriz Inversa A⁻¹:
0.5000  -0.5000  -1.5000  1.5000
-0.5000  1.5000  -1.5000  0.5000
-1.5000  -1.5000  3.5000  -1.5000
1.5000  0.5000  -1.5000  0.5000


* Ejemplo 2

In [33]:
# La matriz A =
A = [
    [4, 4, 5, 1],
    [3, 4, 2, 2],
    [2, 1, 4, 1],
    [3, 2, 5, 4],
]
# tiene como inversa
# A_inv =[[-34.,  31.,  52., -20.],
#         [ 19., -17., -29.,  11.],
#         [ 12., -11., -18.,   7.],
#         [  1.,  -1.,  -2.,   1.]]
A_inv = inv_matrix(A)
print("Matriz Inversa A⁻¹:")
print_matrix(A_inv)

Matriz Inversa A⁻¹:
-34.0000  31.0000  52.0000  -20.0000
19.0000  -17.0000  -29.0000  11.0000
12.0000  -11.0000  -18.0000  7.0000
1.0000  -1.0000  -2.0000  1.0000


## Ejercicios

* Ejercicio 1

In [34]:
A = [[2, -3], [-1, 1]]

A_inv = inv_matrix(A)
print("Matriz Inversa A⁻¹:")
print_matrix(A_inv)

Matriz Inversa A⁻¹:
-1.0000  -3.0000
-1.0000  -2.0000


* Ejercicio 2

In [35]:
A = [
    [4, 0, 0, 5],
    [1, 0, 4, 0],
    [3, 4, 1, 3],
    [1, 3, 3, 0],
]

A_inv = inv_matrix(A)
print("Matriz Inversa A⁻¹:")
print_matrix(A_inv)

Matriz Inversa A⁻¹:
-36.0000  45.0000  60.0000  -80.0000
3.0000  -4.0000  -5.0000  7.0000
9.0000  -11.0000  -15.0000  20.0000
29.0000  -36.0000  -48.0000  64.0000


* Ejercicio 3

In [36]:
A = [
    [0, 0, 0, 0, 0, 0, 1, -1],
    [0, 1, -1, 1, 0, -1, 0, 1],
    [-1, -1, 0, 0, 2, 1, 0, 0],
    [-1, -1, -1, 1, 2, 0, 0, 1],
    [-1, 1, 1, 0, -1, -1, 0, 2],
    [0, 1, 0, 0, -1, -1, 0, 0],
    [1, -1, -1, 1, 2, 1, 0, 2],
    [2, 0, 0, 0, 0, 1, 2, 0],
]

A_inv = inv_matrix(A)
print("Matriz Inversa A⁻¹:")
print_matrix(A_inv)

Matriz Inversa A⁻¹:
2.0000  -1.0000  0.0000  -1.0000  0.0000  2.0000  2.0000  -1.0000
0.0000  1.0000  1.0000  -1.0000  0.0000  0.0000  0.0000  0.0000
6.0000  -1.0000  0.0000  -3.0000  1.0000  1.0000  4.0000  -3.0000
6.0000  1.0000  -1.0000  -3.0000  1.0000  -3.0000  3.0000  -3.0000
2.0000  -1.0000  1.0000  -1.0000  0.0000  3.0000  2.0000  -1.0000
-2.0000  2.0000  -0.0000  0.0000  -0.0000  -4.0000  -2.0000  1.0000
-1.0000  -0.0000  -0.0000  1.0000  -0.0000  -0.0000  -1.0000  1.0000
-2.0000  -0.0000  -0.0000  1.0000  -0.0000  -0.0000  -1.0000  1.0000


* Ejercicio 4

In [37]:
A = [
    [1, 0, 0, 0, -1, 0, 0, -1, 1, -1],
    [1, 1, 0, -1, -1, 1, 0, 0, 1, -1],
    [-1, 0, -1, 0, 0, 0, -1, 1, 0, 0],
    [0, 0, -1, 0, -1, -1, 1, 0, 1, 0],
    [-1, 0, 0, -1, 1, 1, 1, 1, 0, -1],
    [1, 0, 0, 1, -1, -1, -1, 1, -1, 0],
    [1, 1, 1, 0, 1, 0, -1, -1, -1, 1],
    [1, 1, 1, 1, 0, 0, 1, 1, 0, 0],
    [1, 1, 1, 1, 1, 0, -1, -1, 0, 0],
    [0, 0, -1, -1, -1, 0, 1, 1, 1, -1],
]

A_inv = inv_matrix(A)
print("Matriz Inversa A⁻¹:")
print_matrix(A_inv)

Matriz Inversa A⁻¹:
14.0000  -8.0000  9.0000  -4.0000  0.0000  -4.0000  9.0000  7.0000  -8.0000  3.0000
-2.0000  2.0000  -1.0000  2.0000  1.0000  1.0000  -1.0000  -1.0000  1.0000  -2.0000
-27.0000  14.0000  -18.0000  5.0000  -2.0000  7.0000  -17.0000  -13.0000  16.0000  -2.0000
12.0000  -6.0000  8.0000  -2.0000  1.0000  -3.0000  7.0000  6.0000  -7.0000  0.0000
6.0000  -4.0000  4.0000  -2.0000  0.0000  -2.0000  4.0000  3.0000  -3.0000  2.0000
18.0000  -9.0000  12.0000  -4.0000  1.0000  -5.0000  11.0000  9.0000  -11.0000  1.0000
8.0000  -4.0000  5.0000  -1.0000  1.0000  -2.0000  5.0000  4.0000  -5.0000  0.0000
-5.0000  2.0000  -3.0000  0.0000  -1.0000  1.0000  -3.0000  -2.0000  3.0000  1.0000
-11.0000  5.0000  -7.0000  1.0000  -2.0000  2.0000  -7.0000  -5.0000  7.0000  1.0000
1.0000  -1.0000  1.0000  -1.0000  -1.0000  -1.0000  1.0000  1.0000  -1.0000  1.0000
